In [ ]:
from qubic.lib.MapMaking.NeuralNetworkMapMaking.utils.pyro import InvTransmissionPyro, InvTransmissionEstimator
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Qsamplings import equ2gal
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel
import numpy as np
import healpy as hp

# Qubic dict

In [ ]:
dictfilename = "qubic/qubic/dicts/pipeline_demo.dict"
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d["RA_center"], d["DEC_center"])

d["nf_recon"] = 1
d["MultiBand"] = False
d["synthbeam_kmax"] = 1
nf_sub = d["nf_sub"]
d["synthbeam_fraction"] = 1
d["noiseless"] = True
d["photon_noise"] = False
d["use_synthbeam_fits_file"] = False
d["npointings"] = 1000
d["nside"] = 128

# Build Sky

In [ ]:
seed = 3
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
sky_map = np.array(d["nf_sub"] * [hp.synfast(cl_cmb, d["nside"], new=True, verbose=False).T])[0]
print(sky_map.shape)

In [ ]:
hp.mollview(sky_map[..., 0])

# Pyro Modules

In [ ]:
inv_trans_pyro = InvTransmissionPyro()
inv_trans_estimator = InvTransmissionEstimator(d, eta = 0.8, prior_mean = 0.0, prior_sigma = 0.2, lr=3e-3, sigma_noise=1e-18, checkpoint="invT_svi_ckpt.pt")

# Build Dataset

In [ ]:
map_convolve = inv_trans_estimator.convolve_map(sky_map)

N_samples = 5
tod_sky, tod_det = inv_trans_estimator.compute_data_from_sky(map_convolve, N_samples)

# Build model

In [ ]:
inv_trans_estimator.build_model()

# Train

In [ ]:
inv_trans_estimator.train(tod_det, tod_sky, n_steps=400, save_every=20, start=0, use_checkpoint=False)

In [ ]:
mean, std = inv_trans_estimator.posterior(tod_det, num_samples=50)

In [ ]:
print(mean)
print(std)

# Plot

In [ ]:
inv_trans_estimator.plot_results(mean, std, N_samples)